In [1]:
%%capture
pip install ray --upgrade

In [2]:
#Import
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from functools import partial
import os
os.environ["TUNE_MAX_PENDING_TRIALS_PG"] = "1"
import torchvision
from torchvision import datasets, transforms
import random
import ray
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray import air, tune
#ray.shutdown()
#ray.init(object_store_memory=20000000000, num_cpus=16, num_gpus=4)
#os.environ["RAY_ALLOW_SLOW_STORAGE"] = "1"
from torch.nn import Parameter
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
%%capture
pip install pytorch-gradcam

In [4]:
%%capture
pip install grad-cam

In [5]:
%%capture
!pip install "opencv-python-headless<4.3"

In [6]:
%%capture
pip install tqdm

In [7]:
import wandb
wandb.login()

wandb: Currently logged in as: nbennewiz. Use `wandb login --relogin` to force relogin


True

In [8]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.sobel_cam import sobel_cam

In [9]:
import tqdm

In [10]:
#seed everything
seed = 100
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [11]:
def colorMapping(x, name_map="magma"):
    cm = plt.get_cmap(name_map)

    x_map1 = cm(x[ 0, :, :,].detach().numpy())
    
    x_map2 = torch.Tensor(x_map1[ :, :, :3]).to(torch.float).transpose(-2, -1).transpose(0, 1)

    return x_map2


In [12]:
#create transformer
#function to add gaussian noise, because pytorch does not have one, just for gaussian blur
'''
def gauss_noise_tensor(img):
    assert isinstance(img, torch.Tensor)
    dtype = img.dtype
    if not img.is_floating_point():
        img = img.to(torch.float32)
    
    mask = []
    for i in range(3):
        img_part = img[i].to(torch.double)
        mask_part = torch.where(img_part > float(img_part[0, 0]), 1., 0.).to(torch.float)
        mask.append(mask_part)
    
    mask = torch.Tensor(np.array([mask_part.detach().numpy() for mask_part in mask]))
    
    sigma = (0.1**0.8)
    
    out = img + sigma * mask* torch.randn_like(img)
    
    if out.dtype != dtype:
        out = out.to(dtype)
        
    return out
'''
def gauss_noise_tensor(img):
    assert isinstance(img, torch.Tensor)
    dtype = img.dtype
    if not img.is_floating_point():
        img = img.to(torch.float32)
    
    sigma = (0.1**1)
    
    out = img + sigma * torch.randn_like(img)
    
    if out.dtype != dtype:
        out = out.to(dtype)
        
    return out

transform_normal = transforms.Compose([transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                colorMapping
                               ])

transform_gauss = transforms.Compose([
                                gauss_noise_tensor
                               ])

#Load train data
#Get Images loaded
dataset_nonDemented = datasets.ImageFolder("../Alzheimer/Dataset2/NonDemented/", transform=transform_normal)
#get count of instances in dir
num_nonDemented = len(os.listdir(path="./Dataset2/NonDemented/NonDemented"))
#create label data
labels_nonDemented = [np.array([1,0,0,0])] * num_nonDemented
#exchange targets from dataset with selfmade targets
dataset_nonDemented.targets = labels_nonDemented

dataset_veryMildDemented = datasets.ImageFolder("../Alzheimer/Dataset2/VeryMildDemented/", transform=transform_normal)
num_veryMildDemented = len(os.listdir(path="./Dataset2/VeryMildDemented/VeryMildDemented"))
labels_veryMildDemented = [np.array([0,1,0,0])] * num_veryMildDemented
dataset_veryMildDemented.targets = labels_veryMildDemented

dataset_mildDemented = datasets.ImageFolder("../Alzheimer/Dataset2/MildDemented/", transform=transform_normal)
num_mildDemented = len(os.listdir(path="./Dataset2/MildDemented/MildDemented"))
labels_mildDemented = [np.array([0,0,1,0])] * num_mildDemented
dataset_mildDemented.targets = labels_mildDemented

dataset_moderateDemented = datasets.ImageFolder("../Alzheimer/Dataset2/ModerateDemented/", transform=transform_normal)
num_moderateDemented = len(os.listdir(path="./Dataset2/ModerateDemented/ModerateDemented"))
labels_moderateDemented = [np.array([0,0,0,1])] * num_moderateDemented
dataset_moderateDemented.targets = labels_moderateDemented

'''
print("Length of nonDemented-Data_train: ", int(num_nonDemented*(1-test_slice)))
print("Length of veryMildDemented-Data_train: ", int(num_veryMildDemented*(1-test_slice)))
print("Length of mildDemented-Data_train: ", int(num_mildDemented*(1-test_slice)))
print("Length of moderateDemented-Data_train: ", int(num_moderateDemented*(1-test_slice)))
print()
print("Length of nonDemented-Data_test: ", int(num_nonDemented*test_slice))
print("Length of veryMildDemented-Data_test: ", int(num_veryMildDemented*test_slice))
print("Length of mildDemented-Data_test: ", int(num_mildDemented*test_slice))
print("Length of moderateDemented-Data_test: ", int(num_moderateDemented*test_slice))


train_data = []
for i in range(int(num_nonDemented*test_slice), num_nonDemented):
    train_data.append([transform_gauss(dataset_nonDemented[i][0]), labels_nonDemented[i]])
for i in range(int(num_veryMildDemented*test_slice), num_veryMildDemented):
    train_data.append([transform_gauss(dataset_veryMildDemented[i][0]), labels_veryMildDemented[i]])
for i in range(int(num_mildDemented*test_slice), num_mildDemented):
    train_data.append([transform_gauss(dataset_mildDemented[i][0]), labels_mildDemented[i]])
for i in range(int(num_moderateDemented*test_slice), num_moderateDemented):
    train_data.append([transform_gauss(dataset_moderateDemented[i][0]), labels_moderateDemented[i]])
'''
    
train_data = []
for i in range(num_nonDemented):
    train_data.append([dataset_nonDemented[i][0], labels_nonDemented[i]])
for i in range(num_veryMildDemented):
    train_data.append([dataset_veryMildDemented[i][0], labels_veryMildDemented[i]])
for i in range(num_mildDemented):
    train_data.append([dataset_mildDemented[i][0], labels_mildDemented[i]])
for i in range(num_moderateDemented):
    train_data.append([dataset_moderateDemented[i][0], labels_moderateDemented[i]])

In [13]:
#shuffle train_data
random.shuffle(train_data)

#Split train-data into val and train-data
test_slice = 0.1 #20%
test_data = train_data[0:int(len(train_data) * test_slice)]
train_data = train_data[int(len(train_data) * test_slice):]

#Split train-data into val and train-data
val_slice = 0.15 #20%
val_data = train_data[0:int(len(train_data) * val_slice)]
train_data = train_data[int(len(train_data) * val_slice):]

print("Length of training-data: ", len(train_data))
print("Length of validation-data: ",len(val_data))
print("Length of test-data: ", len(test_data))

batch_size = 100

#Create dataset and dataloader
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last = True, num_workers=8)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True, drop_last = True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, drop_last = True, num_workers=8)


Length of training-data:  4896
Length of validation-data:  864
Length of test-data:  640


In [14]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        #init convolutional layers
        self.conv1 = nn.Conv2d(3, 16, 2, 2)#3 input_layers, 16 output_layers, 3 kernel size, 2 step size
        self.conv2 = nn.Conv2d(16, 32, 2, 2)#16 input_layers, 32 output_layers, 3 kernel size, 2 step size
        self.conv3 = nn.Conv2d(32, 64, 2, 2)#32 input_layers, 64 output_layers, 3 kernel size, 2 step size
        self.conv4 = nn.Conv2d(64, 128, 2, 2)#64 input_layers, 256 output_layers, 3 kernel size, 2 step size

        #init batch normalization
        self.norm1 = nn.BatchNorm2d(16, affine=True)#affine= True means with learnable parameters
        self.norm2 = nn.BatchNorm2d(32, affine=True)#affine= True means with learnable parameters
        self.norm3 = nn.BatchNorm2d(64, affine=True)#affine= True means with learnable parameters
        self.norm4 = nn.BatchNorm2d(128, affine=True)#affine= True means with learnable parameters
        
        self.downsample1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(16)
        )
        
        self.downsample2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(32)
        )
        
        self.downsample3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(64)
        )
        
        self.downsample4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(128)
        )
        
    def forward(self, x):
        #Layer 1
        residual = x
        residual  = self.downsample1(residual)
        x = self.conv1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = residual + x
        #print(x.size())
        
        #Layer2
        residual = x
        residual  = self.downsample2(residual)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = residual + x
        #print(x.size())

        #Layer3 
        residual = x
        residual  = self.downsample3(residual)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = residual + x
        #print(x.size())

        #Layer 4
        residual = x
        residual  = self.downsample4(residual)
        x = self.conv4(x)
        x = self.norm4(x)
        x = F.relu(x)
        x = residual + x

        return x



In [15]:
class PatchEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=1, stride=1)

    def forward(self, x):
        x = self.conv(x)

        #Reshape
        x = torch.reshape(x, (x.size(0), x.size(1), x.size(2) * x.size(3)))
        return x

In [16]:
def get_attention(queries, keys, values):
    scale = queries.shape[1] ** -0.5

    attention_scores = (queries @ keys.transpose(-2, -1)) * scale

    attention_probabilities = F.softmax(attention_scores, dim=-1)

    attention = attention_probabilities @ values

    return attention

In [17]:
class QueriesKeysValuesExtractor(nn.Module):
    def __init__(self, token_dim, head_dim, n_heads):
        super().__init__()
        self.head_dim = head_dim
        self.n_heads = n_heads

        queries_keys_values_dim = 3* self.head_dim * self.n_heads
        self.input_to_queries_keys_values = nn.Linear(
            in_features=token_dim, 
            out_features=queries_keys_values_dim, 
            bias=False
        )

    def forward(self, x):
        batch_size, n_tokens, token_dim = x.shape

        queries_keys_values = self.input_to_queries_keys_values(x)

        queries_keys_values = queries_keys_values.reshape(batch_size, 3, self.n_heads, n_tokens, self.head_dim)

        queries, keys, values = queries_keys_values.unbind(dim=1)

        return queries, keys, values

In [18]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, token_dim, head_dim, n_heads, dropout_p):
        super().__init__()
        self.qkv_extractor = QueriesKeysValuesExtractor(
            token_dim=token_dim,
            head_dim=head_dim,
            n_heads = n_heads
        )

        self.concatenated_heads_dim = n_heads * head_dim

        self.attention_to_output = nn.Linear(in_features=self.concatenated_heads_dim, out_features=token_dim)

        self.output_dropout = nn.Dropout(p=dropout_p)

    def forward(self, x, return_attention=False):
        batch_size, n_tokens, token_dim = x.shape

        queries, keys, values = self.qkv_extractor(x)

        attention = get_attention(queries=queries, keys=keys, values=values)
        attention_map = attention.clone()
        attention = attention.reshape(batch_size, n_tokens, self.concatenated_heads_dim)

        x = self.attention_to_output(attention)
        x = self.output_dropout(x)
        
        if return_attention == True:
            return x, attention_map
        else:
            return x

In [19]:
class TransformerBlock(nn.Module):
    def __init__(self, token_dim, mhsa_head_dim, mhsa_n_dim, multilayer_perceptron_dim, dropout_p, out_features):
        super().__init__()
        #init layer
        self.out_features=out_features

        self.layer_norm1 = nn.LayerNorm(normalized_shape=token_dim)

        self.multi_head_self_attention = MultiHeadSelfAttention(
            token_dim=token_dim,
            head_dim = mhsa_head_dim,
            n_heads = mhsa_n_dim,
            dropout_p = dropout_p
        )

        self.layer_norm2 = nn.LayerNorm(normalized_shape=token_dim)

        self.mlp = nn.Sequential(
            nn.Linear(in_features=token_dim, out_features=multilayer_perceptron_dim),
            nn.Dropout(p=dropout_p),
            nn.GELU(),
            nn.Linear(in_features=multilayer_perceptron_dim, out_features=out_features)
            #Maybe add extra dropout?
        )

    def forward(self, x):
        residual = x
        x = self.layer_norm1(x)
        x = self.multi_head_self_attention(x)
        x = x + residual

        residual = x
        x = self.layer_norm2(x)
        x = self.mlp(x)
        if self.out_features != 1:
            x = x + residual

        return x

In [20]:
class Encoder(nn.Module):
    def __init__(self, token_dim, mhsa_head_dim, mhsa_n_dim, multilayer_perceptron_dim, dropout_p, num_layers):
        super().__init__()

        self.transformer_blocks = nn.Sequential()
        
        for i in range(num_layers-1):
            self.transformer_blocks.append(TransformerBlock(token_dim, mhsa_head_dim, mhsa_n_dim, multilayer_perceptron_dim, dropout_p, out_features=token_dim))
            
        self.transformer_blocks.append(TransformerBlock(token_dim, mhsa_head_dim, mhsa_n_dim, multilayer_perceptron_dim, dropout_p, out_features=1))

    def forward(self, x):
        x = self.transformer_blocks(x)

        return x
    
    def get_attention_maps(self, x):
        attention_maps = []
        
        for block in self.transformer_blocks:
            _, attn_map = block.multi_head_self_attention(x, return_attention=True)
            attention_maps.append(attn_map)
            x = block(x)
        
        return attention_maps

In [21]:
class MLPClassifier(nn.Module):
    def __init__(self, in_features, num_classes):
        super().__init__()

        self.layerNorm = nn.LayerNorm(normalized_shape=in_features)

        self.linear = nn.Linear(in_features=in_features, out_features=num_classes)

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):

        x = self.layerNorm(x)

        x = self.linear(x)

        x = self.softmax(x)

        return x

In [22]:
class TokenConcatenator(nn.Module):
    def __init__(self, batch_size):
        super().__init__()

        class_token = torch.zeros((batch_size, 128, 1))
        self.class_token = Parameter(class_token)

    def forward(self, x):

        class_token = self.class_token
        
        x = torch.cat((x, class_token), dim=-1)

        return x

In [23]:
class PositionEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        position_embedding = torch.zeros((1, 128, 197))
        self.position_embedding = Parameter(position_embedding)

    def forward(self, x):
        x = x + self.position_embedding
        return x

In [24]:
class CNN_T(nn.Module):
    def __init__(self, lr, batch_size, head_dim, mhsa_n_dim, multilayer_perceptron_dim, dropout_p, num_layers):
        super(CNN_T, self).__init__()
        #init variables
        self.lr = lr
        self.batch_size = batch_size
        self.head_dim = head_dim
        self.mhsa_n_dim = mhsa_n_dim
        self.multilayer_perceptron_dim = multilayer_perceptron_dim
        self.dropout_p = dropout_p
        self.num_layers = num_layers
        
        #create the model
        #self._create_model()
        
    def create_model(self):
        self.convNet = ConvNet()

        self.patchEmbedding = PatchEmbedding()

        self.tokenConcatenator = TokenConcatenator(batch_size=self.batch_size)

        self.positionEmbedding = PositionEmbedding()

        self.encoder = Encoder(
            token_dim=197,#must stay the same
            mhsa_head_dim=self.head_dim,#can be changed
            mhsa_n_dim=self.mhsa_n_dim,#can be changed
            multilayer_perceptron_dim=self.multilayer_perceptron_dim,#can be changed
            dropout_p=self.dropout_p,#can be changed
            num_layers=self.num_layers#can be changed
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d((64, 1))

        self.mlp_classifier = MLPClassifier(64, 4)
        
    def forward(self, x):

        #convolutional model
        x = self.convNet(x)
        #print(x.size())

        #patch embedding
        x = self.patchEmbedding(x)
        #print(x.size())

        #concat with token
        x = self.tokenConcatenator(x)
        #print(x.size())

        #positional embedding
        x = self.positionEmbedding(x)
        #print(x.size())

        #transformer block
        x = self.encoder(x)
        #print(x.size())
        
        x = self.avgpool(x)
        
        #Reshape out_encoder
        x = torch.reshape(x, (x.size(0), x.size(1)))
        #print(x.size())
        
        
        
        #mlp
        x = self.mlp_classifier(x)
        #print(x.size())

        return x

In [25]:
def train_one_iteration(model, criterion, optimizer, train_loader):
    train_losses = 0
    train_accuracy = 0
    
    for inp_data, labels in train_loader:
        #reset optimizer
        optimizer.zero_grad()
        
        #get data into right shape
        labels = torch.tensor(labels, dtype=torch.float, device =device)
        inp_data = inp_data.to(device).to(torch.float)
        
        #get predictions from the model
        preds = model(inp_data)
        
        #calculate the loss and accuracy
        loss = criterion(labels, preds)
        accuracy = (preds.argmax(dim=-1) == labels.argmax(dim=-1)).float().mean()
        
        #for the average loss
        train_losses += loss.item()
        train_accuracy += accuracy
        
        #calculate the gradients
        loss.backward()
        
        #perform the backpropagation
        optimizer.step()
    
    #calculate the average loss
    avg_loss = train_losses/len(train_loader)
    avg_acc = train_accuracy/len(train_loader)
    
    return avg_loss, avg_acc

In [26]:
def val_one_iteration(model, criterion, val_loader):
    val_losses = 0
    val_accuracy = 0
    
    for inp_data, labels in val_loader:
        #get data into right shape
        labels = torch.tensor(labels, dtype=torch.float, device =device)
        inp_data = inp_data.to(device).to(torch.float)
        
        #get predictions from the model
        preds = model(inp_data)
        
        #calculate the loss and accuracy
        loss = criterion(labels, preds)
        accuracy = (preds.argmax(dim=-1) == labels.argmax(dim=-1)).float().mean()
        
        #for the average loss
        val_losses += loss.item()
        val_accuracy += accuracy
        
    #calculate the average loss
    avg_loss = val_losses/len(val_loader)
    avg_acc = val_accuracy/len(val_loader)
    
    return avg_loss, avg_acc

In [27]:
def train_specific_model(config):
    checkpoint_dir=""
    
    
    #get variables
    lr = config["lr"]
    head_dim = config["head_dim"]
    mhsa_n_dim = config["mhsa_n_dim"]
    multilayer_perceptron_dim = config["multilayer_perceptron_dim"]
    dropout_p = config["dropout_p"]
    num_layers = config["num_layers"]
    
    data_id = config["data_id"]
    model_id = config["model_id"]
    
    #epochs = config["epochs"]
    #get data
    train_data, val_data, test_data = ray.get(data_id)
    
    batch_size = config["batch_size"]
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last = True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True, drop_last = True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, drop_last = True)
    #init the model
    #model = nn.DataParallel(CNN_T(lr, 100, head_dim)).to(device)
    #get and init model
    model = ray.get(model_id)
    model.lr = lr
    model.batch_size = batch_size
    model.head_dim = head_dim
    model.mhsa_n_dim = mhsa_n_dim
    model.multilayer_perceptron_dim = multilayer_perceptron_dim
    model.dropout_p = dropout_p
    model.num_layers = num_layers
    model.create_model()
    model = model.to(device)
    
    wandb.init(project="Optimization_Project", entity="nbennewiz")
    wandb.config = {
        "lr": lr,
        "head_dim": head_dim,
        "mhsa_n_dim": mhsa_n_dim,
        "multilayer_perceptron_dim": multilayer_perceptron_dim,
        "dropout_p": dropout_p,
        "num_layers": num_layers,
        "batch_size": batch_size
    }
    wandb.log({
        "lr": lr,
        "head_dim": head_dim,
        "mhsa_n_dim": mhsa_n_dim,
        "multilayer_perceptron_dim": multilayer_perceptron_dim,
        "dropout_p": dropout_p,
        "num_layers": num_layers,
        "batch_size": batch_size
    })
    
    #init criterion and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr)
    
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(os.path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
    
    #start the training
    for i in range(100):
        #training
        train_losses = 0
        train_accuracy = 0

        for inp_data, labels in train_loader:
            #reset optimizer
            optimizer.zero_grad()

            #get data into right shape
            labels = labels.to(device).to(torch.float)
            inp_data = inp_data.to(device).to(torch.float)

            #get predictions from the model
            preds = model(inp_data)

            #calculate the loss and accuracy
            loss = criterion(labels, preds)
            accuracy = (preds.argmax(dim=-1) == labels.argmax(dim=-1)).cpu().float().mean()

            #for the average loss
            train_losses += loss.item()
            train_accuracy += accuracy

            #calculate the gradients
            loss.backward()

            #perform the backpropagation
            optimizer.step()
        #calculate the average loss
        avg_train_loss = train_losses/len(train_loader)
        avg_train_acc = train_accuracy/len(train_loader)
        
        val_losses = 0
        val_accuracy = 0

        for inp_data, labels in val_loader:
            with torch.no_grad():
                #get data into right shape
                labels = labels.to(device).to(torch.float)
                inp_data = inp_data.to(device).to(torch.float)

                #get predictions from the model
                preds = model(inp_data)

                #calculate the loss and accuracy
                loss = criterion(labels, preds)
                accuracy = (preds.argmax(dim=-1) == labels.argmax(dim=-1)).cpu().float().mean()

                #for the average loss
                val_losses += loss.item()
                val_accuracy += accuracy.item()

        #calculate the average loss
        avg_val_loss = val_losses/len(val_loader)

        avg_val_acc = val_accuracy/len(val_loader)
        
        #validation
        #avg_val_loss, avg_val_acc = val_one_iteration(model, criterion, val_loader)
        with tune.checkpoint_dir(i) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)
        
        wandb.log({
            "epoch": i,
            "train-loss": avg_train_loss,
            "train-accuracy": avg_train_acc,
            "val-loss": avg_val_loss,
            "val-accuracy": avg_val_acc
        })
        tune.report(loss=avg_val_loss, accuracy=avg_val_acc)
        #print("loss=", avg_val_loss, "accuracy=", avg_val_acc)
    
    test_losses = 0
    test_accuracy = 0

    for inp_data, labels in test_loader:
        with torch.no_grad():
            #get data into right shape
            labels = labels.to(device).to(torch.float)
            inp_data = inp_data.to(device).to(torch.float)

            #get predictions from the model
            preds = model(inp_data)

            #calculate the loss and accuracy
            loss = criterion(labels, preds)
            accuracy = (preds.argmax(dim=-1) == labels.argmax(dim=-1)).cpu().float().mean()

            #for the average loss
            test_losses += loss.item()
            test_accuracy += accuracy.item()

    #calculate the average loss
    avg_test_loss = test_losses/len(test_loader)
    avg_test_acc = test_accuracy/len(test_loader)
    wandb.log({
        "test-loss": avg_test_loss,
        "test-accuracy": avg_test_acc
    })
    
    wandb.finish()
    #print("Finished Training Model")

In [28]:
"""
model = CNN_T(0.001, 100, 26, 2, 2, 0.0)
model_id = ray.put(model)

data_id = ray.put((train_data, val_data))

#write config
config = {
    "lr": 0.0001,
    "head_dim": 56,
    "mhsa_n_dim": 10,
    "multilayer_perceptron_dim": 100,
    "dropout_p":0.01,
    "num_layers": 3,
    "data_id": data_id,
    "model_id": model_id
}
    

train_specific_model(config)
"""

'\nmodel = CNN_T(0.001, 100, 26, 2, 2, 0.0)\nmodel_id = ray.put(model)\n\ndata_id = ray.put((train_data, val_data))\n\n#write config\nconfig = {\n    "lr": 0.0001,\n    "head_dim": 56,\n    "mhsa_n_dim": 10,\n    "multilayer_perceptron_dim": 100,\n    "dropout_p":0.01,\n    "num_layers": 3,\n    "data_id": data_id,\n    "model_id": model_id\n}\n    \n\ntrain_specific_model(config)\n'

In [29]:
def optimization(data_id, num_samples=100, max_num_epochs=100, gpus_per_trial=1):
    #init model and safe with ray
    model = CNN_T(0.001, 100, 26, 2, 2, 0.0, 2)
    model_id = ray.put(model)
    
    #write config
    config = {
        "lr": tune.loguniform(1e-4, 1e-1),
        "head_dim": tune.randint(2, 84),
        "mhsa_n_dim": tune.randint(1, 24),
        "multilayer_perceptron_dim": tune.randint(1, 124),
        "dropout_p":tune.loguniform(1e-3, 0.2),
        "num_layers":tune.randint(1, 10),
        "batch_size": tune.choice([16, 32, 64, 100]),
        "data_id": data_id,
        "model_id": model_id
    }
    TUNE_MAX_PENDING_TRIALS_PG=1
    #init scheduler
    scheduler = ASHAScheduler(
        metric="accuracy",
        mode="max",
        max_t=max_num_epochs,
        grace_period=5,
        reduction_factor=2
    )

    #init reporter
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    print("here1")
    #run the hyperparameter optimization
    result = tune.run(
        partial(train_specific_model),
        resources_per_trial={"cpu": 16, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
        checkpoint_at_end=True
    )

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(best_trial.last_result["accuracy"]))
    
    return result

In [30]:
#store data
data_id = ray.put((train_data, val_data, test_data))


2023-01-19 12:54:19,239	WARNING services.py:1732 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 1059069952 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-01-19 12:54:19,947	INFO worker.py:1538 -- Started a local Ray instance.


In [31]:

result = optimization(data_id)


here1
== Status ==
Current time: 2023-01-19 12:56:05 (running for 00:00:00.18)
Memory usage on this node: 162.9/1007.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None | Iter 10.000: None | Iter 5.000: None
Resources requested: 16.0/256 CPUs, 1.0/4 GPUs, 0.0/917.62 GiB heap, 0.0/9.31 GiB objects (0.0/1.0 accelerator_type:A100)
Result logdir: /root/ray_results/train_specific_model_2023-01-19_12-56-04
Number of trials: 1/100 (1 RUNNING)
+----------------------------------+----------+--------------------+--------------+-------------+------------+------------+--------------+------------------------+--------------+
| Trial name                       | status   | loc                |   batch_size |   dropout_p |   head_dim |         lr |   mhsa_n_dim |   multilayer_perceptro |   num_layers |
|                                  |          |                    |              |             |            |            |              |      

(func pid=2586447) wandb: Currently logged in as: nbennewiz. Use `wandb login --relogin` to force relogin
(func pid=2586743) wandb: Currently logged in as: nbennewiz. Use `wandb login --relogin` to force relogin


== Status ==
Current time: 2023-01-19 12:56:19 (running for 00:00:14.31)
Memory usage on this node: 190.8/1007.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None | Iter 10.000: None | Iter 5.000: None
Resources requested: 64.0/256 CPUs, 4.0/4 GPUs, 0.0/917.62 GiB heap, 0.0/9.31 GiB objects (0.0/1.0 accelerator_type:A100)
Result logdir: /root/ray_results/train_specific_model_2023-01-19_12-56-04
Number of trials: 5/100 (1 PENDING, 4 RUNNING)
+----------------------------------+----------+--------------------+--------------+-------------+------------+-------------+--------------+------------------------+--------------+
| Trial name                       | status   | loc                |   batch_size |   dropout_p |   head_dim |          lr |   mhsa_n_dim |   multilayer_perceptro |   num_layers |
|                                  |          |                    |              |             |            |             |             

(func pid=2586447) wandb: wandb version 0.13.9 is available!  To upgrade, please run:
(func pid=2586447) wandb:  $ pip install wandb --upgrade
(func pid=2586447) wandb: Tracking run with wandb version 0.13.7
(func pid=2586447) wandb: Run data is saved locally in /root/ray_results/train_specific_model_2023-01-19_12-56-04/train_specific_model_a247f_00000_0_batch_size=64,dropout_p=0.0163,head_dim=69,lr=0.0043,mhsa_n_dim=8,multilayer_perceptron_dim=88,_2023-01-19_12-56-05/wandb/run-20230119_125615-30se67tp
(func pid=2586447) wandb: Run `wandb offline` to turn off syncing.
(func pid=2586447) wandb: Syncing run fresh-spaceship-1
(func pid=2586447) wandb: ⭐️ View project at https://wandb.ai/nbennewiz/Optimization_Project
(func pid=2586447) wandb: 🚀 View run at https://wandb.ai/nbennewiz/Optimization_Project/runs/30se67tp
(func pid=2586447) /opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py:146: UserWarning: The given NumPy array is not writable, and PyTorch does not sup

== Status ==
Current time: 2023-01-19 12:56:24 (running for 00:00:19.31)
Memory usage on this node: 194.1/1007.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None | Iter 10.000: None | Iter 5.000: None
Resources requested: 64.0/256 CPUs, 4.0/4 GPUs, 0.0/917.62 GiB heap, 0.0/9.31 GiB objects (0.0/1.0 accelerator_type:A100)
Result logdir: /root/ray_results/train_specific_model_2023-01-19_12-56-04
Number of trials: 5/100 (1 PENDING, 4 RUNNING)
+----------------------------------+----------+--------------------+--------------+-------------+------------+-------------+--------------+------------------------+--------------+
| Trial name                       | status   | loc                |   batch_size |   dropout_p |   head_dim |          lr |   mhsa_n_dim |   multilayer_perceptro |   num_layers |
|                                  |          |                    |              |             |            |             |             

...

== Status ==
Current time: 2023-01-19 13:35:31 (running for 00:39:26.22)
Memory usage on this node: 184.5/1007.7 GiB 
Using AsyncHyperBand: num_stopped=98
Bracket: Iter 80.000: 0.989360754985755 | Iter 40.000: 0.987500011920929 | Iter 20.000: 0.9850000143051147 | Iter 10.000: 0.9781650641025641 | Iter 5.000: 0.5450000092387199
Resources requested: 32.0/256 CPUs, 2.0/4 GPUs, 0.0/917.62 GiB heap, 0.0/9.31 GiB objects (0.0/1.0 accelerator_type:A100)
Result logdir: /root/ray_results/train_specific_model_2023-01-19_12-56-04
Number of trials: 100/100 (2 RUNNING, 98 TERMINATED)
+----------------------------------+------------+--------------------+--------------+-------------+------------+-------------+--------------+------------------------+--------------+------------+------------+----------------------+
| Trial name                       | status     | loc                |   batch_size |   dropout_p |   head_dim |          lr |   mhsa_n_dim |   multilayer_perceptro |   num_layers |       los

wandb: ERROR Failed to sample metric: process no longer exists (pid=2621632)


== Status ==
Current time: 2023-01-19 13:35:57 (running for 00:39:52.70)
Memory usage on this node: 176.6/1007.7 GiB 
Using AsyncHyperBand: num_stopped=99
Bracket: Iter 80.000: 0.989360754985755 | Iter 40.000: 0.987500011920929 | Iter 20.000: 0.9850000143051147 | Iter 10.000: 0.9781650641025641 | Iter 5.000: 0.5473842638786193
Resources requested: 16.0/256 CPUs, 1.0/4 GPUs, 0.0/917.62 GiB heap, 0.0/9.31 GiB objects (0.0/1.0 accelerator_type:A100)
Result logdir: /root/ray_results/train_specific_model_2023-01-19_12-56-04
Number of trials: 100/100 (1 RUNNING, 99 TERMINATED)
+----------------------------------+------------+--------------------+--------------+-------------+------------+-------------+--------------+------------------------+--------------+------------+------------+----------------------+
| Trial name                       | status     | loc                |   batch_size |   dropout_p |   head_dim |          lr |   mhsa_n_dim |   multilayer_perceptro |   num_layers |       los

2023-01-19 13:36:10,339	INFO tune.py:762 -- Total run time: 2405.67 seconds (2405.30 seconds for the tuning loop).


== Status ==
Current time: 2023-01-19 13:36:10 (running for 00:40:05.32)
Memory usage on this node: 176.7/1007.7 GiB 
Using AsyncHyperBand: num_stopped=100
Bracket: Iter 80.000: 0.989360754985755 | Iter 40.000: 0.987500011920929 | Iter 20.000: 0.9850000143051147 | Iter 10.000: 0.9771634615384616 | Iter 5.000: 0.5473842638786193
Resources requested: 0/256 CPUs, 0/4 GPUs, 0.0/917.62 GiB heap, 0.0/9.31 GiB objects (0.0/1.0 accelerator_type:A100)
Result logdir: /root/ray_results/train_specific_model_2023-01-19_12-56-04
Number of trials: 100/100 (100 TERMINATED)
+----------------------------------+------------+--------------------+--------------+-------------+------------+-------------+--------------+------------------------+--------------+------------+------------+----------------------+
| Trial name                       | status     | loc                |   batch_size |   dropout_p |   head_dim |          lr |   mhsa_n_dim |   multilayer_perceptro |   num_layers |       loss |   accuracy

In [32]:
#testing to train one specific model
#data = (train_data, val_data, test_data)
#train_specific_model(config, data)

In [33]:
best_trial = result.get_best_trial(metric="accuracy", mode="max", scope="last")
print(best_trial.config)

{'lr': 0.00045259141640607885, 'head_dim': 39, 'mhsa_n_dim': 20, 'multilayer_perceptron_dim': 28, 'dropout_p': 0.15801372157434743, 'num_layers': 2, 'batch_size': 16, 'data_id': ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000001000000), 'model_id': ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000002000000)}


In [34]:
print(best_trial.last_result)

{'loss': 0.002303776475304437, 'accuracy': 0.9953703703703703, 'time_this_iter_s': 6.512756824493408, 'should_checkpoint': True, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 100, 'trial_id': 'a247f_00039', 'experiment_id': 'b9751f01f0b742059653af67190413a6', 'date': '2023-01-19_13-22-31', 'timestamp': 1674134551, 'time_total_s': 709.2951195240021, 'pid': 2600642, 'hostname': '1047936aaad1', 'node_ip': '172.17.0.2', 'config': {'lr': 0.00045259141640607885, 'head_dim': 39, 'mhsa_n_dim': 20, 'multilayer_perceptron_dim': 28, 'dropout_p': 0.15801372157434743, 'num_layers': 2, 'batch_size': 16, 'data_id': ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000001000000), 'model_id': ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000002000000)}, 'time_since_restore': 709.2951195240021, 'timesteps_since_restore': 0, 'iterations_since_restore': 100, 'warmup_time': 0.0035810470581054688, 'experiment_tag': '39_batch_size=16,dropout_p=0.1580,head_di

wandb: ERROR Failed to sample metric: process no longer exists (pid=2623017)


In [52]:
print(best_trial.checkpoint)

<_TrackedCheckpoint id=99 storage='PERSISTENT' dir_or_data=/root/ray_results/train_specific_model_2023-01-19_12-56-04/train_specific_model_a247f_00039_39_batch_size=16,dropout_p=0.1580,head_dim=39,lr=0.0005,mhsa_n_dim=20,multilayer_perceptron_dim=2_2023-01-19_13-10-39/checkpoint_000099>


In [50]:
#init random parameter model
best_trained_model = CNN_T(0.001, 100, 26, 2, 2, 0.0, 2)

#get parameter out of the config
lr = best_trial.config["lr"]
head_dim = best_trial.config["head_dim"]
mhsa_n_dim = best_trial.config["mhsa_n_dim"]
multilayer_perceptron_dim = best_trial.config["multilayer_perceptron_dim"]
dropout_p = best_trial.config["dropout_p"]
num_layers = best_trial.config["num_layers"]
batch_size = best_trial.config["batch_size"]

#change parameters accordingly
best_trained_model.lr = lr
best_trained_model.batch_size = batch_size
best_trained_model.head_dim = head_dim
best_trained_model.mhsa_n_dim = mhsa_n_dim
best_trained_model.multilayer_perceptron_dim = multilayer_perceptron_dim
best_trained_model.dropout_p = dropout_p
best_trained_model.num_layers = num_layers
best_trained_model.create_model()
best_trained_model = best_trained_model.to(device)

best_checkpoint_dir = best_trial.checkpoint
model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint"))
best_trained_model.load_state_dict(model_state)

TypeError: expected str, bytes or os.PathLike object, not _TrackedCheckpoint

In [56]:
def train_specific_model(config):
    checkpoint_dir=""
    
    
    #get variables
    lr = config["lr"]
    head_dim = config["head_dim"]
    mhsa_n_dim = config["mhsa_n_dim"]
    multilayer_perceptron_dim = config["multilayer_perceptron_dim"]
    dropout_p = config["dropout_p"]
    num_layers = config["num_layers"]
    
    data_id = config["data_id"]
    model_id = config["model_id"]
    
    #epochs = config["epochs"]
    #get data
    train_data, val_data, test_data = ray.get(data_id)
    
    batch_size = config["batch_size"]
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last = True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True, drop_last = True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, drop_last = True)
    #init the model
    #model = nn.DataParallel(CNN_T(lr, 100, head_dim)).to(device)
    #get and init model
    model = ray.get(model_id)
    model.lr = lr
    model.batch_size = batch_size
    model.head_dim = head_dim
    model.mhsa_n_dim = mhsa_n_dim
    model.multilayer_perceptron_dim = multilayer_perceptron_dim
    model.dropout_p = dropout_p
    model.num_layers = num_layers
    model.create_model()
    model = model.to(device)
    
    wandb.init(project="Optimization_Project", entity="nbennewiz")
    wandb.config = {
        "lr": lr,
        "head_dim": head_dim,
        "mhsa_n_dim": mhsa_n_dim,
        "multilayer_perceptron_dim": multilayer_perceptron_dim,
        "dropout_p": dropout_p,
        "num_layers": num_layers,
        "batch_size": batch_size
    }
    wandb.log({
        "lr": lr,
        "head_dim": head_dim,
        "mhsa_n_dim": mhsa_n_dim,
        "multilayer_perceptron_dim": multilayer_perceptron_dim,
        "dropout_p": dropout_p,
        "num_layers": num_layers,
        "batch_size": batch_size
    })
    
    #init criterion and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr)

    #start the training
    for i in range(100):
        #training
        train_losses = 0
        train_accuracy = 0

        for inp_data, labels in train_loader:
            #reset optimizer
            optimizer.zero_grad()

            #get data into right shape
            labels = labels.to(device).to(torch.float)
            inp_data = inp_data.to(device).to(torch.float)

            #get predictions from the model
            preds = model(inp_data)

            #calculate the loss and accuracy
            loss = criterion(labels, preds)
            accuracy = (preds.argmax(dim=-1) == labels.argmax(dim=-1)).cpu().float().mean()

            #for the average loss
            train_losses += loss.item()
            train_accuracy += accuracy

            #calculate the gradients
            loss.backward()

            #perform the backpropagation
            optimizer.step()
        #calculate the average loss
        avg_train_loss = train_losses/len(train_loader)
        avg_train_acc = train_accuracy/len(train_loader)
        
        val_losses = 0
        val_accuracy = 0

        for inp_data, labels in val_loader:
            with torch.no_grad():
                #get data into right shape
                labels = labels.to(device).to(torch.float)
                inp_data = inp_data.to(device).to(torch.float)

                #get predictions from the model
                preds = model(inp_data)

                #calculate the loss and accuracy
                loss = criterion(labels, preds)
                accuracy = (preds.argmax(dim=-1) == labels.argmax(dim=-1)).cpu().float().mean()

                #for the average loss
                val_losses += loss.item()
                val_accuracy += accuracy.item()

        #calculate the average loss
        avg_val_loss = val_losses/len(val_loader)

        avg_val_acc = val_accuracy/len(val_loader)
        
        #validation
        #avg_val_loss, avg_val_acc = val_one_iteration(model, criterion, val_loader)

        wandb.log({
            "epoch": i,
            "train-loss": avg_train_loss,
            "train-accuracy": avg_train_acc,
            "val-loss": avg_val_loss,
            "val-accuracy": avg_val_acc
        })
        #print("loss=", avg_val_loss, "accuracy=", avg_val_acc)
    
    test_losses = 0
    test_accuracy = 0

    for inp_data, labels in test_loader:
        with torch.no_grad():
            #get data into right shape
            labels = labels.to(device).to(torch.float)
            inp_data = inp_data.to(device).to(torch.float)

            #get predictions from the model
            preds = model(inp_data)

            #calculate the loss and accuracy
            loss = criterion(labels, preds)
            accuracy = (preds.argmax(dim=-1) == labels.argmax(dim=-1)).cpu().float().mean()

            #for the average loss
            test_losses += loss.item()
            test_accuracy += accuracy.item()

    #calculate the average loss
    avg_test_loss = test_losses/len(test_loader)
    avg_test_acc = test_accuracy/len(test_loader)
    wandb.log({
        "test-loss": avg_test_loss,
        "test-accuracy": avg_test_acc
    })
    
    wandb.finish()
    return model
    #print("Finished Training Model")

In [59]:

model = CNN_T(0.001, 100, 26, 2, 2, 0.0, 2)
model_id = ray.put(model)

data_id = ray.put((train_data, val_data, test_data))


config = best_trial.config
config["model_id"] = model_id
config["data_id"] = data_id
model = train_specific_model(config)



(raylet) Spilled 10295 MiB, 6 objects, write throughput 211 MiB/s.


batch_size,▁
dropout_p,▁
head_dim,▁
lr,▁
mhsa_n_dim,▁
multilayer_perceptron_dim,▁
num_layers,▁
batch_size,16
dropout_p,0.15801
head_dim,39
lr,0.00045


batch_size,▁
dropout_p,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
head_dim,▁
lr,▁
mhsa_n_dim,▁
multilayer_perceptron_dim,▁
num_layers,▁
test-accuracy,▁
test-loss,▁
train-accuracy,▁▇██████████████████████████████████████


NameError: name 'config' is not defined